In [1]:
import tweepy
from twarc import Twarc2, expansions
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import nltk
from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
from nltk.stem.porter import PorterStemmer as PS
import time
import schedule
import datetime
import gc

In [ ]:
#lemmatization
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence

In [ ]:
import stanza

nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')
doc = nlp('Her bags are better than his.')
print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')

In [ ]:
#lemmatization
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def lemma(txt):
    s = ''
    doc = nlp(txt)
    
    for sent in doc.sentences:
        for word in sent.words:
            s += word.lemma + ' '
    
    return s

In [ ]:
df1 = pd.read_csv('./tweet-sentiment/2021-01_slist.csv',usecols = ['Compound'])
df2 = pd.read_csv('./tweet-sentiment/2021-01_slist_Number0.csv', usecols = ['Compound'])

In [ ]:
df1["bool"] = (df1['Compound'] == df2['Compound'])

In [ ]:
df_comp = df1.compare(df2)
df_comp

In [ ]:
#del df3
gc.collect()

In [11]:
df = pd.read_csv('./binance/BTCUSDT/1d/BTCUSDT-1d-2021-01.csv',usecols = ['unix','open','close'])
df.head(31)

,unix,open,close
0,1609459200,28923.63,29331.69
1,1609545600,29331.70,32178.33
2,1609632000,32176.45,33000.05
3,1609718400,33000.05,31988.71
4,1609804800,31989.75,33949.53
5,1609891200,33949.53,36769.36
6,1609977600,36769.36,39432.28
7,1610064000,39432.48,40582.81
8,1610150400,40586.96,40088.22
9,1610236800,40088.22,38150.02


In [16]:
df1 = df.pct_change()
print(df1['open'])

0          NaN
1     0.014109
2     0.096986
3     0.025596
4    -0.030615
5     0.061263
6     0.083059
7     0.072428
8     0.029277
9    -0.012288
10   -0.048348
11   -0.071813
12   -0.038441
13    0.097572
14    0.047465
15   -0.061509
16   -0.020210
17   -0.004723
18    0.022260
19   -0.019674
20   -0.012080
21   -0.130151
22    0.068002
23   -0.026464
24    0.005657
25   -0.000163
26    0.006505
27   -0.064743
28    0.099004
29    0.026315
30    0.000485
Name: open, dtype: float64


In [ ]:
df1.head(100)

In [4]:
print(df.diff())

       unix     open    close
0       NaN      NaN      NaN
1   86400.0   408.07  2846.64
2   86400.0  2844.75   821.72
3   86400.0   823.60 -1011.34
4   86400.0 -1010.30  1960.82
5   86400.0  1959.78  2819.83
6   86400.0  2819.83  2662.92
7   86400.0  2663.12  1150.53
8   86400.0  1154.48  -494.59
9   86400.0  -498.74 -1938.20
10  86400.0 -1938.20 -2745.55
11  86400.0 -2739.65 -1353.23
12  86400.0 -1361.22  3320.14
13  86400.0  3322.23  1773.12
14  86400.0  1773.83 -2402.28
15  86400.0 -2407.78  -747.24
16  86400.0  -742.45  -166.37
17  86400.0  -169.99   802.66
18  86400.0   797.47  -739.78
19  86400.0  -720.52  -423.26
20  86400.0  -433.71 -4618.10
21  86400.0 -4616.24  2095.04
22  86400.0  2098.01  -867.17
23  86400.0  -872.00   181.90
24  86400.0   181.45    -5.70
25  86400.0    -5.26   213.57
26  86400.0   209.82 -2101.62
27  86400.0 -2101.82  2998.71
28  86400.0  3005.99   887.34
29  86400.0   878.10    10.68
30  86400.0    16.61 -1169.90


In [ ]:
a=1612051200000
b=1611964800000
print((a-b)/24,(a-b)/(24*3600))

In [ ]:
import pandas as pd
print (pd.__file__)

In [21]:
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-01.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-02.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-03.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-04.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-05.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
with zipfile.ZipFile('binance/BTCUSDT/15m/BTCUSDT-15m-2021-06.zip') as existing_zip:
    existing_zip.extractall('binance/BTCUSDT/15m')
